In [0]:

csv_data ="""transaction_id,customer_name,region,product,category,quantity,unit_price,date
1,Rajesh,North,Laptop,Electronics,1,55000,2024-01-12
2,Sneha,West,Refrigerator,Electronics,1,32000,2024-02-05
3,Anil,South,Shampoo,Personal Care,5,150,2024-01-17
4,Divya,North,Mobile,Electronics,2,20000,2024-03-22
5,Vikram,East,Washing Machine,Electronics,1,28000,2024-02-28
6,Preeti,West,Sneakers,Fashion,2,4000,2024-01-31
7,Aman,South,TV,Electronics,1,45000,2024-02-15
8,Isha,North,Notebook,Stationery,10,60,2024-01-10
9,Kunal,East,Pencil,Stationery,20,10,2024-03-05
10,Tanvi,West,Face Cream,Personal Care,3,200,2024-03-19
"""

dbutils.fs.put("dbfs:/tmp/sales_transactions.csv", csv_data, overwrite=True)


Wrote 603 bytes.


True

In [0]:
py_df=spark.read.csv("dbfs:/tmp/sales_transactions.csv", header=True)
display(py_df)
py_df.write.mode("overwrite").parquet("dbfs:/output/sales.parquet")
py_df.write.format("delta").mode("overwrite").save("dbfs:/output2/sales_transactions")

transaction_id,customer_name,region,product,category,quantity,unit_price,date
1,Rajesh,North,Laptop,Electronics,1,55000,2024-01-12
2,Sneha,West,Refrigerator,Electronics,1,32000,2024-02-05
3,Anil,South,Shampoo,Personal Care,5,150,2024-01-17
4,Divya,North,Mobile,Electronics,2,20000,2024-03-22
5,Vikram,East,Washing Machine,Electronics,1,28000,2024-02-28
6,Preeti,West,Sneakers,Fashion,2,4000,2024-01-31
7,Aman,South,TV,Electronics,1,45000,2024-02-15
8,Isha,North,Notebook,Stationery,10,60,2024-01-10
9,Kunal,East,Pencil,Stationery,20,10,2024-03-05
10,Tanvi,West,Face Cream,Personal Care,3,200,2024-03-19


In [0]:
# Switch to local Hive metastore (not Unity Catalog)
spark.sql("USE hive_metastore.default")
delta_path = f"dbfs:/output2/sales_transactions"
# Create table
spark.sql("DROP TABLE IF EXISTS sales_transactions")
spark.sql(f"""
    CREATE TABLE sales_transactions
    USING DELTA
    LOCATION '{delta_path}'
""")


DataFrame[]

In [0]:
spark.sql("""select*from sales_transactions""")

DataFrame[transaction_id: string, customer_name: string, region: string, product: string, category: string, quantity: string, unit_price: string, date: string]

Data transformation

In [0]:
from pyspark.sql.functions import expr,month,date_format
transformed_df=spark.table("sales_transactions").withColumn("total_price",expr("quantity*unit_price")).withColumn('month', month('date')).withColumn("formated_date",date_format('date',"dd-MM-yyyy")).withColumn("is_high_value",expr("case when total_price>30000 then True else False end"))

display(transformed_df)
transformed_df.write.format("delta").mode("overwrite").save("dbfs:/output2/sales_transformed")





transaction_id,customer_name,region,product,category,quantity,unit_price,date,total_price,month,formated_date,is_high_value
1,Rajesh,North,Laptop,Electronics,1,55000,2024-01-12,55000.0,1,12-01-2024,true
2,Sneha,West,Refrigerator,Electronics,1,32000,2024-02-05,32000.0,2,05-02-2024,true
3,Anil,South,Shampoo,Personal Care,5,150,2024-01-17,750.0,1,17-01-2024,false
4,Divya,North,Mobile,Electronics,2,20000,2024-03-22,40000.0,3,22-03-2024,true
5,Vikram,East,Washing Machine,Electronics,1,28000,2024-02-28,28000.0,2,28-02-2024,false
6,Preeti,West,Sneakers,Fashion,2,4000,2024-01-31,8000.0,1,31-01-2024,false
7,Aman,South,TV,Electronics,1,45000,2024-02-15,45000.0,2,15-02-2024,true
8,Isha,North,Notebook,Stationery,10,60,2024-01-10,600.0,1,10-01-2024,false
9,Kunal,East,Pencil,Stationery,20,10,2024-03-05,200.0,3,05-03-2024,false
10,Tanvi,West,Face Cream,Personal Care,3,200,2024-03-19,600.0,3,19-03-2024,false


Aggregation and insight

In [0]:
from pyspark.sql.functions import sum
df=spark.read.format("delta").load("dbfs:/output2/sales_transformed")
grouped_transcations=df.groupBy("region").count()
display(grouped_transcations)
grouped_salesamount=df.groupBy("category").sum("total_price")
display(grouped_salesamount)
month_trend=df.groupBy("month").sum("total_price")
display(month_trend)
high_purchased=df.orderBy("total_price",ascending=False).limit(1)
display(high_purchased)
q1=df.filter((df.month>=1) & (df.month<4))
q1_total_revenue=q1.agg(sum("total_price"))
display(q1_total_revenue)


region,count
South,2
East,2
West,3
North,3


category,sum(total_price)
Stationery,800.0
Fashion,8000.0
Electronics,200000.0
Personal Care,1350.0


month,sum(total_price)
1,64350.0
3,40800.0
2,105000.0


transaction_id,customer_name,region,product,category,quantity,unit_price,date,total_price,month,formated_date,is_high_value
1,Rajesh,North,Laptop,Electronics,1,55000,2024-01-12,55000.0,1,12-01-2024,true


sum(total_price)
210150.0


 Update & Delete Scenarios


In [0]:
from delta.tables import DeltaTable
deltable=DeltaTable.forPath(spark,"dbfs:/output2/sales_transformed")
deltable.update(condition="category='Sationery'",set={"unit_price":"unit_price*0.10"})
display(deltable.toDF())

transaction_id,customer_name,region,product,category,quantity,unit_price,date,total_price,month,formated_date,is_high_value
1,Rajesh,North,Laptop,Electronics,1,55000,2024-01-12,55000.0,1,12-01-2024,true
2,Sneha,West,Refrigerator,Electronics,1,32000,2024-02-05,32000.0,2,05-02-2024,true
3,Anil,South,Shampoo,Personal Care,5,150,2024-01-17,750.0,1,17-01-2024,false
4,Divya,North,Mobile,Electronics,2,20000,2024-03-22,40000.0,3,22-03-2024,true
5,Vikram,East,Washing Machine,Electronics,1,28000,2024-02-28,28000.0,2,28-02-2024,false
6,Preeti,West,Sneakers,Fashion,2,4000,2024-01-31,8000.0,1,31-01-2024,false
7,Aman,South,TV,Electronics,1,45000,2024-02-15,45000.0,2,15-02-2024,true
8,Isha,North,Notebook,Stationery,10,60,2024-01-10,600.0,1,10-01-2024,false
9,Kunal,East,Pencil,Stationery,20,10,2024-03-05,200.0,3,05-03-2024,false
10,Tanvi,West,Face Cream,Personal Care,3,200,2024-03-19,600.0,3,19-03-2024,false


In [0]:
deltable.delete(condition="quantity < 3")
display(deltable.toDF())

transaction_id,customer_name,region,product,category,quantity,unit_price,date,total_price,month,formated_date,is_high_value
3,Anil,South,Shampoo,Personal Care,5,150,2024-01-17,750.0,1,17-01-2024,false
8,Isha,North,Notebook,Stationery,10,60,2024-01-10,600.0,1,10-01-2024,false
9,Kunal,East,Pencil,Stationery,20,10,2024-03-05,200.0,3,05-03-2024,false
10,Tanvi,West,Face Cream,Personal Care,3,200,2024-03-19,600.0,3,19-03-2024,false


In [0]:
from pyspark.sql.functions import current_date
from datetime import date
cuurrent_month=date.today().month
formatted_date=date.today().strftime("%d-%m-%Y")
new_row = spark.createDataFrame([(11, "Dhanu", "west", "blue-jeans", "clothings", 1, 1500, date.today(), 1500,cuurrent_month,formatted_date,"false")], ["transaction_id", "customer_name", "region", "product", "category", "quantity", "unit_price", "date", "total_price","month","formated_date","is_high_value"])
display(new_row)
deltable.alias("t").merge(new_row.alias("s"),"t.transaction_id=s.transaction_id").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
display(deltable.toDF())
deltable.toDF().write.format("delta").mode("overwrite").save("dbfs:/output2/sales_transformed")


transaction_id,customer_name,region,product,category,quantity,unit_price,date,total_price,month,formated_date,is_high_value
11,Dhanu,west,blue-jeans,clothings,1,1500,2025-08-08,1500,8,08-08-2025,false


transaction_id,customer_name,region,product,category,quantity,unit_price,date,total_price,month,formated_date,is_high_value
3,Anil,South,Shampoo,Personal Care,5,150,2024-01-17,750.0,1,17-01-2024,false
8,Isha,North,Notebook,Stationery,10,60,2024-01-10,600.0,1,10-01-2024,false
9,Kunal,East,Pencil,Stationery,20,10,2024-03-05,200.0,3,05-03-2024,false
10,Tanvi,West,Face Cream,Personal Care,3,200,2024-03-19,600.0,3,19-03-2024,false
11,Dhanu,west,blue-jeans,clothings,1,1500,2025-08-08,1500.0,8,08-08-2025,false


partitioning & Optimization (Bonus)

In [0]:
deltable.toDF().write.format("delta").mode("overwrite").partitionBy("region").save("dbfs:/output2/regions")
deltable.toDF().write.format("delta").mode("overwrite").partitionBy("month").save("dbfs:/output2/month")


In [0]:
#optimization using z-oredering
spark.sql("OPTIMIZE delta.`/output2/sales_transformed` ZORDER BY (region)")

spark.sql("OPTIMIZE delta.`/output2/sales_transformed` ZORDER BY (month)")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

In [0]:

spark.read.format("delta") .load("dbfs:/output2/sales_transformed") .filter("region = 'West'") .explain(True)

== Parsed Logical Plan ==
'Filter ('region = West)
+- Relation [transaction_id#17054,customer_name#17055,region#17056,product#17057,category#17058,quantity#17059,unit_price#17060,date#17061,total_price#17062,month#17063,formated_date#17064,is_high_value#17065] parquet

== Analyzed Logical Plan ==
transaction_id: string, customer_name: string, region: string, product: string, category: string, quantity: string, unit_price: string, date: string, total_price: double, month: int, formated_date: string, is_high_value: boolean
Filter (region#17056 = West)
+- Relation [transaction_id#17054,customer_name#17055,region#17056,product#17057,category#17058,quantity#17059,unit_price#17060,date#17061,total_price#17062,month#17063,formated_date#17064,is_high_value#17065] parquet

== Optimized Logical Plan ==
Filter (isnotnull(region#17056) AND (region#17056 = West))
+- Relation [transaction_id#17054,customer_name#17055,region#17056,product#17057,category#17058,quantity#17059,unit_price#17060,date#1706

In [0]:
# Capture your optimize calls
df_region = spark.sql(
  "OPTIMIZE delta.`/output2/sales_transformed` ZORDER BY (region)"
)
df_month = spark.sql(
  "OPTIMIZE delta.`/output2/sales_transformed` ZORDER BY (month)"
)

# View the first few rows of each result
df_region.show(truncate=False)
df_month.show(truncate=False)

+-------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|path                           |metrics                                                                                                                                                                                                                                   |
+-------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|dbfs:/output2/sales_transformed|{0, 0, {NULL, NULL, 0.0, 0, 0}, {NULL, NULL, 0.0, 0, 0}, 0, {minCubeSize(107374182400), {1, 3290}, {0, 0}, 1, {0, 0}, 0, NULL}, NULL, 0, 0, 1, 1, false, 0, 0, 1